# [GPT-Assign04] "Retrieval-Augmented Generation”
-----
### Lecture06: RAG

* Implement a complete **RAG pipeline with a Stuff Documents chain**.
* You must implement the chain manually.
Give a **`ConversationBufferMemory`** to the chain.
* Use this **document** to perform RAG: https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223
* Ask the following questions to the chain:
    * Is Aaronson guilty?
    * What message did he write in the table?
    * Who is Julia?

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory
# from langchain.chains import ConversationChain



llm = ChatOpenAI(
    temperature=0.1,
)

# memory= ConversationBufferMemory(return_messages=True)

memory = ConversationBufferMemory(
    llm=llm,
    return_messages=True,
    memory_key="history",
)

# memort = ConversationChain(
#     llm=llm, 
#     verbose=True, 
#     memory=memory
# )

cache_dir= LocalFileStore("/workspaces/FULLSTACK-GPT/.cache")


splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator= "\n",
    chunk_size= 600,
    chunk_overlap= 100,
)

loader = UnstructuredFileLoader("/workspaces/FULLSTACK-GPT/files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant. Answer questions using only the context. 
    If you don't know the answer, just say you don't know; don't make it up.":\n\n{context}"""),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}")
])

def load_memory(input):       
    return memory.load_memory_variables({})["history"]

chain = {"context": retriever,
         "question": RunnablePassthrough(), 
         "history":  RunnableLambda(load_memory),} | prompt | llm



# def invoke_chain(question):     
#     result= chain.invoke({"question": question})
#     memory.save_context({"input": question}, {"output": result.content},)
#     print(result)

def invoke_chain(question):
    result = chain.invoke(question).content
    memory.save_context(
        {"input": question},
        {"output": result},
    )
    print(f"Result: {result}")
   

In [2]:
invoke_chain("Is Aaronson guilty?")

Result: Yes, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.


In [3]:
invoke_chain("What message did he write in the table?")

Result: He wrote "2+2=5" on the table.


In [4]:
invoke_chain("Who is Julia?v")

Result: Julia is a person who Winston loves and cares for deeply.
